In [ ]:
import numpy as np
import xarray as xr
import xesmf as xe

In [ ]:
ds_in = xr.open_dataset("/home/sia/FjordsSim_data/oslofjord/Grid/OF160_grid_v2.nc")

In [ ]:
ds_in

In [ ]:
print(f"Lat min: {ds_in.lat_rho.values.min()}")
print(f"Lat max: {ds_in.lat_rho.values.max()}")
print(f"Lon min: {ds_in.lon_rho.values.min()}")
print(f"Lon max: {ds_in.lon_rho.values.max()}")

In [ ]:
ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.linspace(59.1, 59.98, num=440), {"units": "degrees_north"}),
        "lon": (["lon"], np.linspace(10.2, 10.85, num=66), {"units": "degrees_east"}),
    }
)

In [ ]:
ds_out

In [ ]:
regridder = xe.Regridder(ds_in.rename({"lon_rho": "lon", "lat_rho": "lat"}), ds_out, "bilinear", unmapped_to_nan=True)

In [ ]:
da = regridder(ds_in["h"])

In [ ]:
da_h = ds_in["h"]

In [ ]:
da_h.plot()

In [ ]:
da = da.where(da > 5, np.nan)
da.plot()

In [ ]:
xr.Dataset({"h": da.fillna(5)}).to_netcdf("/home/sia/FjordsSim_data/oslofjord/OF_inner_66to440_bathymetry.nc")